### Importação das Bibliotecas necessarias 

In [1]:
import sqlite3
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime

### Configurações Basicas
 - Definindo a conexão com o Banco de Dados
 - Configurando a visualização do Pandas

In [2]:
CONN = sqlite3.connect("./database.db")
CURSOR = CONN.cursor()
pd.set_option('display.width', 1000)

### Visualização de Itens parados no Estoque

In [8]:
estoque = pd.read_sql("SELECT * FROM TempoEmEstoque", CONN)
print(estoque)

   ID_Produto        Nome_Produto  Quantidade_em_Estoque Ultima_Compra  Dias_em_Estoque
0           1        Smartphone X                    150    2024-04-01            431.0
1           7    Leite Longa Vida                    400    2024-04-10            422.0
2           8    Calça Jeans Azul                    200    2024-04-15            417.0
3           4  O Pequeno Príncipe                    100    2024-04-20            412.0
4          10        Limpa Vidros                    180    2024-04-25            407.0
5          12    Raquete de Tênis                     50    2024-05-25            377.0
6          16        Câmera de Ré                     60    2024-05-28            374.0
7          18   Semente de Alface                    300    2024-06-01            370.0
8          11         Pneu Aro 15                     80    2024-06-03            368.0
9          13  Adubo Orgânico 1kg                    150    2024-06-06            365.0


### Promoções Ativas

In [9]:
promocoes = pd.read_sql("SELECT * FROM vw_Promocao_Ativa", CONN)
print(promocoes)

   PromocaoID  PercentualDesconto  DataInicio     DataFim  ProdutoID           NomeProduto  PrecoOriginal  PrecoPromocional
0           1                  10  2025-06-01  2025-06-30          1          Smartphone X         1200.5           1080.45
1           2                  15  2025-06-05  2025-06-25          3      Camiseta Algodão           49.9             42.41
2           8                   5  2025-06-05  2025-06-12         15  Protetor Solar FPS50           60.0             57.00


### Gráfico de Vendas

In [ ]:
inicioVenda = widgets.DatePicker(
    description='Intervalo de Vendas:',
    disabled=False,
    # Define o valor inicial como a data de hoje, garantindo que haja um valor padrão
    value=datetime.date.today(),
    min=datetime.date(2020, 1, 1), # Data mínima para o seletor
    max=datetime.date.today()      # Data máxima para o seletor
)
fimVenda = widgets.DatePicker(
    description='Até:', 
    disabled=False,
    value=datetime.date.today(),
    min=datetime.date(2020, 1, 1),
    max=datetime.date.today()
)

output_area_vendas = widgets.Output() 

def drawGraphVendas(): 
    with output_area_vendas:
        clear_output(wait=True)

        query_vendas_view = f"""
        SELECT
            Data_Venda,
            Total
        FROM
            vw_Item_Venda_Detalhado
        WHERE
            Data_Venda BETWEEN '{inicioVenda.value}' AND '{fimVenda.value}'
        ORDER BY
            Data_Venda;
        """
        df_vendas_detalhe = pd.read_sql_query(query_vendas_view, CONN)

        if df_vendas_detalhe.empty:
            print("Não há dados nesse intervalo :(")
            return

        df_vendas_detalhe['Data_Venda'] = pd.to_datetime(df_vendas_detalhe['Data_Venda'])

        df_vendas_diarias_agrupadas = df_vendas_detalhe.groupby('Data_Venda')['Total'].sum().reset_index()

        # 3. Gerar todas as datas no intervalo selecionado
        all_dates_vendas = pd.date_range(start=inicioVenda.value, end=fimVenda.value, freq='D')
        df_all_dates_vendas = pd.DataFrame(all_dates_vendas, columns=['Data'])

        df_final_plot_vendas = pd.merge(df_all_dates_vendas, df_vendas_diarias_agrupadas,
                                        left_on='Data', right_on='Data_Venda', how='left')

        df_final_plot_vendas['TotalVendasDiarias'] = df_final_plot_vendas['Total'].fillna(0)
        df_final_plot_vendas = df_final_plot_vendas[['Data', 'TotalVendasDiarias']]

        fig = px.line(df_final_plot_vendas,
                      x='Data',
                      y='TotalVendasDiarias',
                      title=f'Total de Vendas Diárias de {inicioVenda.value} a {fimVenda.value}',
                      labels={'Data': 'Data da Venda', 'TotalVendasDiarias': 'Valor Total das Vendas (R$)'})

        fig.update_xaxes(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )

        fig.show()

def on_date_changeVenda(change):
    drawGraphVendas()

inicioVenda.observe(on_date_changeVenda, names='value')
fimVenda.observe(on_date_changeVenda, names='value')
display(inicioVenda, fimVenda, output_area_vendas)

DatePicker(value=datetime.date(2025, 6, 6), description='Intervalo de Vendas:', max=datetime.date(2025, 6, 6),…

DatePicker(value=datetime.date(2025, 6, 6), description='Até:', max=datetime.date(2025, 6, 6), min=datetime.da…

Output()

### Gráfico de Compras


In [ ]:
inicioCompra = widgets.DatePicker(
    description='Intervalo de:',
    disabled=False,
    value=datetime.date.today(),
    min=datetime.date(2020, 1, 1),
    max=datetime.date.today()
)
fimCompra = widgets.DatePicker(
    disabled=False,
    value=datetime.date.today(),
    min=datetime.date(2020, 1, 1),
    max=datetime.date.today()
)

output_area = widgets.Output()

def drawGraphCompra():
    with output_area:
        clear_output(wait=True)

        query_compras_view = f"""
        SELECT
            DataCompra,
            ValorTotalInformado
        FROM
            vw_Compra_Detalhada
        WHERE
            DataCompra BETWEEN '{inicioCompra.value}' AND '{fimCompra.value}'
        ORDER BY
            DataCompra;
        """
        df_compras_detalhe = pd.read_sql_query(query_compras_view, CONN)

        if df_compras_detalhe.empty:
            print("Não há dados nesse intervalo :(")
            return

        df_compras_detalhe['DataCompra'] = pd.to_datetime(df_compras_detalhe['DataCompra'])

        df_compras_diarias_agrupadas = df_compras_detalhe.groupby('DataCompra')['ValorTotalInformado'].sum().reset_index()

        all_dates = pd.date_range(start=inicioCompra.value, end=fimCompra.value, freq='D')
        df_all_dates = pd.DataFrame(all_dates, columns=['Data'])

        df_final_plot = pd.merge(df_all_dates, df_compras_diarias_agrupadas,
                                 left_on='Data', right_on='DataCompra', how='left')

        df_final_plot['TotalCompras'] = df_final_plot['ValorTotalInformado'].fillna(0)
        df_final_plot = df_final_plot[['Data', 'TotalCompras']] 
        fig = px.line(df_final_plot,
                      x='Data',
                      y='TotalCompras',
                      title=f'Total de Compras Diárias de {inicioCompra.value} a {fimCompra.value}',
                      labels={'Data': 'Data da Compra', 'TotalCompras': 'Valor Total das Compras (R$)'})

        fig.update_xaxes(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )

        fig.show()

def on_date_changeCompra(change):
    drawGraphCompra()

inicioCompra.observe(on_date_changeCompra, names='value')
fimCompra.observe(on_date_changeCompra, names='value')
display(inicioCompra, fimCompra, output_area)

DatePicker(value=datetime.date(2025, 6, 6), description='Intervalo de:', max=datetime.date(2025, 6, 6), min=da…

DatePicker(value=datetime.date(2025, 6, 6), max=datetime.date(2025, 6, 6), min=datetime.date(2020, 1, 1), step…

Output()

### Métodos de Pagamento Utilizados

In [12]:
query_metodo_pag = "SELECT * FROM MetodoPag;"
df_metodo_pag = pd.read_sql_query(query_metodo_pag, CONN)

if df_metodo_pag.empty:
    print("Não foram encontrados registros de pagamento")
else:
    fig = px.pie(df_metodo_pag,
                 names='Metodo',
                 values='Vezes_Usado',
                 title='Frequência de Métodos de Pagamento',
                 hole=0.3, 
                 labels={'Metodo': 'Método de Pagamento', 'Vezes_Usado': 'Vezes Usado'})

    fig.update_traces(textposition='inside', textinfo='percent+label') 
    fig.show()

### Funcionarios por Setor

In [13]:
query_distribuicao_setor = "SELECT * FROM vw_Funcionarios_Ativos_Por_Setor;"
df_distribuicao_setor = pd.read_sql_query(query_distribuicao_setor, CONN)

if df_distribuicao_setor.empty:
    print("Dados não encontrados")
else:
    fig = px.bar(df_distribuicao_setor,
                 x='NomeSetor',
                 y='TotalFuncionarios',
                 title='Distribuição de Funcionários por Setor (Atualmente Ativos)',
                 labels={'NomeSetor': 'Setor', 'TotalFuncionarios': 'Número de Funcionários'},
                 color='NomeSetor', 
                 text='TotalFuncionarios') 

    fig.update_layout(xaxis_tickangle=-45) 
    fig.show()

### Produtos mais Vendidos

In [ ]:
inicio_prod_vendidos = widgets.DatePicker(
    description='Início (Produtos):',
    disabled=False,
    value=datetime.date(2024, 1, 1), # Data de início padrão
    min=datetime.date(2020, 1, 1),
    max=datetime.date.today()
)
fim_prod_vendidos = widgets.DatePicker(
    description='Fim (Produtos):',
    disabled=False,
    value=datetime.date.today(),
    min=datetime.date(2020, 1, 1),
    max=datetime.date.today()
)

top_n_produtos = widgets.IntSlider(
    value=10,
    min=1,
    max=20, 
    step=1,
    description='Top N Produtos:',
    continuous_update=False
)

output_area_prod_vendidos = widgets.Output()

def drawGraphProdutosMaisVendidos():
    with output_area_prod_vendidos:
        clear_output(wait=True) 

        data_inicio = inicio_prod_vendidos.value.strftime('%Y-%m-%d')
        data_fim = fim_prod_vendidos.value.strftime('%Y-%m-%d')
        num_top_n = top_n_produtos.value

        query = f"""
        SELECT
            NomeProduto,
            CategoriaProduto,
            QuantidadeTotalVendida,
            ValorTotalVendido
        FROM vw_Produtos_Mais_Vendidos_Por_Periodo
        WHERE DataVenda BETWEEN '{data_inicio}' AND '{data_fim}';
        """
        df_vendas_periodo = pd.read_sql_query(query, CONN)

        if df_vendas_periodo.empty:
            print("Nenhum produto vendido no intervalo selecionado.")
            return

        df_produtos_agrupados = df_vendas_periodo.groupby(['NomeProduto', 'CategoriaProduto']).agg(
            TotalQuantidade=('QuantidadeTotalVendida', 'sum'),
            TotalValor=('ValorTotalVendido', 'sum')
        ).reset_index()

        df_top_n = df_produtos_agrupados.sort_values(by='TotalQuantidade', ascending=False).head(num_top_n)

        fig = px.bar(df_top_n,
                     x='TotalQuantidade',
                     y='NomeProduto',
                     color='CategoriaProduto', 
                     title=f'Top {num_top_n} Produtos Mais Vendidos ({data_inicio} a {data_fim})',
                     labels={'TotalQuantidade': 'Quantidade Total Vendida', 'NomeProduto': 'Produto'},
                     orientation='h') 

        fig.update_layout(yaxis={'categoryorder':'total ascending'}) 
        fig.show()

def on_param_change(change):
    drawGraphProdutosMaisVendidos()

inicio_prod_vendidos.observe(on_param_change, names='value')
fim_prod_vendidos.observe(on_param_change, names='value')
top_n_produtos.observe(on_param_change, names='value') 

display(inicio_prod_vendidos, fim_prod_vendidos, top_n_produtos, output_area_prod_vendidos)


DatePicker(value=datetime.date(2024, 1, 1), description='Início (Produtos):', max=datetime.date(2025, 6, 6), m…

DatePicker(value=datetime.date(2025, 6, 6), description='Fim (Produtos):', max=datetime.date(2025, 6, 6), min=…

IntSlider(value=10, continuous_update=False, description='Top N Produtos:', max=20, min=1)

Output()

### Fechar conexão com o banco de dados

In [8]:
CONN.close()